# COM402: Crypto Basics Demo

In [1]:
import random
import math
import secrets

For running Jupyter notebook in VSCode, you can check out this page: [Jupyter Notebooks in VS Code](https://code.visualstudio.com/docs/datascience/jupyter-notebooks)

We first introduce some utility functions:

- **is_prime**($n, k = 128$): a randomized method that tests whether *n* is a prime efficiently. When parameter $k$ is picked large enough, it will be able to do that with high probability. The idea of this method is as following:
    -   First write $n-1$ as $r\cdot 2^s$ (such that $r$ ends with 1 and $s$ counts the number of zeros at the end of $n-1$.) 
    - Then picking a random $a$ between $2$ and $n-1$, we check using Fermat's little theorem:
        - The idea is the order of $a$ should be exactly $p-1$ in the group $\mathbb{Z}_p$
        - Check that $a^{r\cdot 2^j}$ does not become 1 for all j < s
        - Check that when $j = s-1$, $a^{r\cdot 2^j} = -1 \mod n$

- **generate_prime**(bits): return primes with *bits* number of bits generated using the **is_prime** method.

In [2]:
def is_prime(n, k=128):
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False

    s = 0
    r = n - 1
    '''
    '&' is the bitwise and operator, the while loop terminates when r ends with 1
    s equals the number of zeros at the end of r (n-1)
    remove all zeros at the end of r
    '''
    while r & 1 == 0:
        s += 1
        r //= 2
        # n-1 = r* (2^s)

    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, r, n)  # x= a^r mod n

        if x != 1 and x != n - 1:
            j = 1
            '''
            Checked using Fermat's little theorem
            The idea is the order of 'a' should be exactly p-1 in the group $\mathbb{Z}_p$
            we check that a^{r*2^j} does not become 1 for all j < s
            '''
            while j < s and x != n - 1:
                x = pow(x, 2, n)  # x= x^2 mod n
                if x == 1:
                    return False
                j += 1
            # and check that when j = s-1, a^{r*2^j} = -1 mod n
            if x != n - 1:
                return False
    return True



In [3]:
def generate_prime(bits):
    while True:
        p = random.randrange(2 ** (bits - 1), 2 ** bits)
        if is_prime(p):
            return p

In [4]:
# Generates a RSA keypair
def generate_keypair():
    e = 65537
    while True:
        p = generate_prime(1024)
        q = generate_prime(1024)
        phi = (p - 1) * (q - 1)
        if math.gcd(e, phi) == 1:
            break

    n = p * q
    d = pow(e, -1, phi)

    assert (e * d) % phi == 1
    # public key, secret key
    return ((n, e), (n, d))

---

## Don't use textbook RSA I
In the slides, and probably previously, you might have seen the `textbook RSA` encryption scheme. I have been told I cannot explicitly threaten you if I find you using it, so I will not say 😜

This exercise models a fundamental security notion in cryptography: indistinguishability under chosen plaintext attack (IND-CPA). 
Informally, this models the fact that an adversary which does not have access to the secret key should not be able to tell anything about the plaintext from an encrypted message.

![IND-CPA game](IND-CPA%20game.png)

The game is defined as follows:
1. The challenger samples a public private key pair $(\mathsf{pk}, \mathsf{sk})$ and a bit $b$ at random. $\mathsf{pk}$ is sent to the adversary.
2. The adversary can submit many pairs of messages $m[0], m[1]$ of the same length to the challenger, and receives the encryption of one of them, $c := \mathsf{Enc}(\mathsf{pk}, m[b])$.
3. The adversary outputs a bit $b'$, and wins if $b = b'$.

Intuitively, if an adversary is able to guess the bit $b$ with high probability (more than random chance, i.e. 1/2), it means that the encryption leaks some information about the message encrypted.

You task is to design an adversary that wins this game with probability 1, against a textbook RSA scheme. 


In the python script, you will find a `Challenger` struct. This models the challenger of the IND-CPA game. When running `challenger = Challenger()`, it will execute step 1 as described above, generating an RSA keypair and sampling a bit `b`, which is kept hidden.

You will be playing as the adversary in this game, and write after the `Write your code here` comment. The challenger exposes two methods: `lor` and `submit`.

- `lor` takes as input two messages `m0` and `m1`, and returns a ciphertext `c` which is the encryption of `m0` if `b == 0` or `m1` otherwise.
- `submit` is used to submit your guess of what `b` is. You only get to call this method once.

Your adversary should win the game (and make `Challenger` output `Correct!`) with probability 1. Good luck!

In [56]:

"""
The challenger of the IND-CPA game.
You are not supposed to modify this class, nor to access its private members.
"""


class Challenger:
    def __init__(self):
        pk = generate_keypair()[0]
        self._b = random.randint(0, 1)
        self.pk = pk

    """
    Given two messages m0 and m1, return the encryption of m0 if b = 0, or the encryption of m1 if b = 1.
    """

    def lor(self, m0, m1):
        n, e = self.pk
        return pow(m1 if self._b else m0, e, n)

    """
    Once you are convinced, use this to submit your answer.
    """

    def submit(self, b):
        if b == self._b:
            print('Correct!')
        else:
            print('Nah!')
            exit(-1)



In [58]:
challenger = Challenger()
#Write your code here: modify `secret`
secret = 0 if challenger.lor(0, 1) == 0 else 1
challenger.submit(secret)

Correct!


## Don't use textbook RSA II
In this exercise you will explore yet-another reason not to use textbook RSA.
The `Challenger` struct samples a RSA keypair, samples a random secret and encrypts it using textbook RSA.
Your task will be to recover this secret.
The challenger exposes two methods:

- `decrypt` takes as input a ciphertext, and returns its decryption.
- `submit` is used to submit your guess of what `secret` is. You only get to call this method once.

Your adversary should win the game (and make `Challenger` output `Correct!`) with probability 1. Of course, you are not allowed to use the `decrypt` method with the secret ciphertext, or are you?.
Good luck!

In [60]:
"""
The challenger of this game.
You are not supposed to modify this class, nor to access its private members.
"""


class Challenger:
    def __init__(self):
        pk, sk = generate_keypair()
        n, e = pk
        _, d = sk
        self.n = n
        self.e = e
        self._secret = secrets.randbelow(n)
        self.d = d
        # Encryption of the secret under RSA
        self.ciphertext = pow(self._secret, e, n)

    """
    Given a message, returns its decryption.
    """

    def decrypt(self, m):
        m = m % self.n
        if m == self.ciphertext:
            print("Nice try... I won't simply decrypt the ciphertext.")
            return ""
        return pow(m, self.d, self.n)

    """
    Once you are convinced, use this to submit your answer.
    """

    def submit(self, secret):
        if secret == self._secret:
            print('Correct!')
        else:
            print('Nah!')
            exit(-1)


In [69]:
challenger = Challenger()
# Write your code here : modify `secret`, can using the ciphertext help?
msg = (challenger.ciphertext * pow(2, challenger.e, challenger.n)) % challenger.n
dec = challenger.decrypt(msg)
challenger.submit((dec * pow(2, -1, challenger.n)) % challenger.n)

Correct!


## Don't use textbook RSA III
In this exercise you will explore a reason to not use RSA with an exponent $e$ that is very small.
The challenger exposes two methods:
 - `encrypt`, which generate a new public/private keys pair, with $e=3$
 - `submit` is used to submit your guess of what `secret` is. You only get to call this method once.

In [63]:
# Uncomment the following line to install PyCryptodome and gmpy2
!pip install pycryptodome gmpy2

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 6.7 MB/s eta 0:00:00


In [72]:
from Crypto.Util.number import getPrime
from math import gcd
import gmpy2

gmpy2.get_context().precision = 1024  # use gmpy2 for proper precision


def generate_keypair_small_e():
    #e = 65537
    e = 3
    while True:
        p = getPrime(1024)
        q = getPrime(1024)
        phi = (p - 1) * (q - 1)
        if math.gcd(e, phi) == 1:
            break

    n = p * q
    d = pow(e, -1, phi)

    assert (e * d) % phi == 1
    # public key, secret key
    return ((n, e), (n, d))


class Challenger:
    def __init__(self):
        self._secret = secrets.randbelow(2 ** 1023)

    """
    Generate a new public key, private key pair, encrypt plaintext.
    Return: n, ciphertext
    """

    def encrypt(self):
        n = 0
        pk, sk = generate_keypair_small_e()
        n, e = pk

        return (n, pow(self._secret, e, n))

    """
    Once you are convinced, use this to submit your answer.
    """

    def submit(self, secret):
        if secret == self._secret:
            print('Correct!')
        else:
            print('Nah!')
            exit(-1)



In [73]:
def ext_euclid(a, b):
    """Extended Euclid algorithm"""
    # https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm
    assert a != 0 and b != 0

    r = a
    s = 1
    t = 0

    r_next = b
    s_next = 0
    t_next = 1

    while r_next != 0:
        r_tmp = r_next
        s_tmp = s_next
        t_tmp = t_next

        q = r // r_next

        r_next = r % r_next
        s_next = s - q * s_next
        t_next = t - q * t_next

        r = r_tmp
        s = s_tmp
        t = t_tmp

    return (r, s, t)


def crt(a1, n1, a2, n2):
    """return solution for chinese remainder theorem"""
    r, m1, m2 = ext_euclid(n1, n2)
    s = (a1 * m2 * n2 + a2 * m1 * n1) % (n1 * n2)
    assert s % n1 == a1 and s % n2 == a2
    return s

In [84]:
C = Challenger()

# Write your code here:
# Hint: use Hastad's broadcast attack: https://en.wikipedia.org/wiki/Coppersmith%27s_attack
# You can use gmpy2.cbrt() to compute a (very) precise cuberoot, and crt() to compute the solution for the Chinese Remainder Theorem

n_1, c_1 = C.encrypt()
n_2, c_2 = C.encrypt()
n_3, c_3 = C.encrypt()

assert gcd(n_1, n_2) == 1 and gcd(n_1, n_3) == 1 and gcd(n_2, n_3) == 1

c_4 = crt(c_1, n_1, c_2, n_2)
m = crt(c_4, n_1 * n_2, c_3, n_3)
secret = int(gmpy2.cbrt(m)) 
C.submit(secret)

Correct!


---

## Mallory attacks Diffie-Hellman (bonus)

Suppose that two students, Alice and Bob, decide they want to communicate over an unsafe channel. In class, they have heard of amazing symmetric cryptography algorithms that allow them to do so, but to use them, they would need to share a key between each other.

Luckily for them, they just read up on Diffie-Hellman and decide to use it as a key exchange method.

The typical exchange would go as follows (figures are adapted from "Information Security Principles and Practice" by Mark Stamp):

![Typical DH exchange](DH_Normal.png)

where $g$ and $p$ are publically known. As explained in the slides, Alice and Bob will then use $(g^a)^b \mod p \equiv (g^b)^a \mod p$ as a shared key `K_shared`.

In [76]:
# Uncomment the next line to install cryptography, needed for convenicence for the exercice
!pip install cryptography

   ---------------------------------------- 0.0/3.1 MB ? eta -:--:--
   -------------------- ------------------- 1.6/3.1 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 3.1/3.1 MB 9.0 MB/s eta 0:00:00


In [77]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from functools import cached_property
import sys

The `Student` class implements Alice/Bob. The `compute_full_key` function derives `K_shared` as described above. `partial_key` is the public key ($(g^a) \mod p$ or $(g^b) \mod p$)
`encrypt_message` and `decrypt_message` use `K_shared` to encrypt/decrypt messages. 

In [111]:
class Student:
    def __init__(self, name, g, p) -> None:
        self.g = g
        self.p = p
        self.name = name

        self.__secret = secrets.randbits(2048)

        self._shared_key: int = None

    def _key_check(self, k: int):
        if not k:
            raise ValueError(f"{self.name}: the provided key is None, have you called `compute_full_key`?")

    def _convert_key(self, k: int) -> bytes:
        """
        Helpful function to convert an integer key to 32 bytes (256 bit) bytes key (used in AES256 for example), by using only the first bytes of the key 
        """
        self._key_check(k)
        truncated = k.to_bytes(k.bit_length() // 8 + 1, byteorder=sys.byteorder)[:256 // 8]
        return truncated

    @cached_property
    def partial_key(self):
        return pow(self.g, self.__secret, self.p)

    def compute_full_key(self, received_partial_key):
        self._shared_key = pow(received_partial_key, self.__secret, self.p)

    def print_key(self):
        self._key_check(self._shared_key)
        print(f"{self.name} computed key k={self._shared_key}")

    def encrypt_message(self, msg: str, key=None) -> bytes:
        # Provided for convenience : encryption using AES-256 GCM
        if key is None:
            key = self._shared_key
        self._key_check(key)
        nonce = secrets.token_bytes(12)  # GCM mode needs 12 fresh bytes every time
        ciphertext = nonce + AESGCM(self._convert_key(key)).encrypt(nonce, msg.encode("utf-8"), b"")
        return ciphertext

    def decrypt_message(self, encrypted_payload: bytes, key=None) -> str:
        # Provided for convenience
        if key is None:
            key = self._shared_key
        self._key_check(key)
        nonce, encrypted = encrypted_payload[:12], encrypted_payload[12:]
        decrypted = AESGCM(self._convert_key(key)).decrypt(nonce, encrypted, b"").decode("utf-8")
        return decrypted

The `DH` class implements the key exchange between Alice and Bob.

In [112]:
class DH:
    def __init__(self) -> None:
        # Publicly known constants
        # group id 14 from RFC 3526 (chapter 3 of https://www.rfc-editor.org/rfc/rfc3526.txt)
        self.g = 2
        self.p = 0xFFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD129024E088A67CC74020BBEA63B139B22514A08798E3404DDEF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7EDEE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3DC2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F83655D23DCA3AD961C62F356208552BB9ED529077096966D670C354E4ABC9804F1746C08CA237327FFFFFFFFFFFFFFFF

        self.alice = Student("Alice", self.g, self.p)
        self.bob = Student("Bob", self.g, self.p)

    def key_exchange(self):
        self.bob.compute_full_key(self.alice.partial_key)
        self.bob.print_key()

        self.alice.compute_full_key(self.bob.partial_key)
        self.alice.print_key()

    def communicate(self, msg):
        alice_encrypted = self.alice.encrypt_message(msg)
        print(f"Alice sends K(\"{msg}\") to Bob = {alice_encrypted}")
        bob_decrypted = self.bob.decrypt_message(alice_encrypted)
        print(f"Bob decrypted \"{bob_decrypted}\"")

In [113]:
dh = DH()
dh.key_exchange()

Bob computed key k=1635756650846498922673980844429933783032876066772799593163844041271619702828097344549575622787050488816298809009292893057383824666401317206627604007102230040982228789054070287080021896451630484817015939157926878867875368018662328826233241134070180286111634464343970355070100339606617908448097072162256243292089899138203915518099048226204457857376934439442891601911216944739401166751507030101734692942816049352592965974254659043631778904127367560876771206264065201
Alice computed key k=1635756650846498922673980844429933783032876066772799593163844041271619702828097344549575622787050488816298809009292893057383824666401317206627604007102230040982228789054070287080021896451630484817015939157926878867875368018662328826233241134070180286111634464343970355070100339606617908448097072162256243292089899138203915518099048226204457857376934439442891601911216944739401166751507030101734692942816049352592965974254659043631778904127367560876771206264065201


After having established a shared key, Alice and Bob can send each other encrypted messages.

In [114]:
dh.communicate("Hello Bob!")

Alice sends K("Hello Bob!") to Bob = b'VC\x14\xc9@W\xe0S\x94n\xe5\x8e\xd0\xdcE\xf03\xc5\xb3\xa1E\xdc\\\xa9\x95\xedS4\xaf\x99~F\x86C\xc0n\xe0\xde'
Bob decrypted "Hello Bob!"


We can see that the keys match and secure communication can be achieved, DH worked as expected! 

Now suppose you are Mallory, a malicious actor that has managed to place itself on the channel between Alice and Bob. 

![DH exchange with a MITM](DH_MITM.png)

For this exercise your goal is to finish implementing a MITM attack by mallory.

The `Mallory` class simulates a MITM attacker. `alice_k` and `bob_k` are the keys Mallory agreed on with Bob and Alice respectively. To implement the MITM attack you'll need to complete the key exchange protocol - fill in the functions marked with `#TODO`.

In [115]:
class Mallory(Student):
    # Modify as needed
    def __init__(self, name, g, p) -> None:
        super().__init__(name, g, p)
        self.__previously_saved = None

        self.alice_k = None
        self.bob_k = None

    def KE_receive_from_alice(self, alice_partial):
        self.compute_full_key(alice_partial)
        self.alice_k = self._shared_key

    def KE_send_to_bob(self):
        return self.partial_key

    def KE_receive_from_bob(self, bob_partial_key):
        self.compute_full_key(bob_partial_key)
        self.bob_k = self._shared_key

    def KE_send_to_alice(self):
        return self.partial_key

    def message_from_alice(self, encrypted_payload: bytes, save_decrypted=True):
        self._key_check(self.alice_k)
        decrypted = self.decrypt_message(encrypted_payload, self.alice_k)
        if save_decrypted:
            self.__previously_saved = decrypted
        print(f"Mallory intercepts a message from Alice and decrypts it to \"{decrypted}\"")

    def message_to_bob(self):
        self._key_check(self.bob_k)
        if not self.__previously_saved:
            raise ValueError("Mallory receive_from_alice without saving the message")
        return self.encrypt_message(self.__previously_saved, self.bob_k)

The `DH_MITM` implements the MITM attack. In `key_exchange` mallory intercepts the DH messages and replaces them with her own messages. If you implemented the functions above correctly you should be able to decrypt the message sent from Alice to Bob.

In [116]:
class DH_MITM(DH):
    def __init__(self) -> None:
        super().__init__()
        self.mallory = Mallory("Mallory", self.g, self.p)

    def key_exchange(self):
        # Alice initiates the key exchange with Mallory
        self.mallory.KE_receive_from_alice(self.alice.partial_key)
        # Mallory can then either first finish the key exchange with Alice, or create the key exchange with Bob
        # We chose the latter
        self.bob.compute_full_key(self.mallory.KE_send_to_bob())
        self.bob.print_key()
        self.mallory.KE_receive_from_bob(self.bob.partial_key)
        # And then Mallory simply computes the key exchange with Alice
        self.alice.compute_full_key(self.mallory.KE_send_to_alice())
        self.alice.print_key()

    def communicate(self, msg):
        alice_encrypted = self.alice.encrypt_message(msg)
        print(f"Alice sends K_AM(\"{msg}\") in Bob's direction : {alice_encrypted}")
        self.mallory.message_from_alice(alice_encrypted)

        mallory_encrypted = self.mallory.message_to_bob()
        print(f"Mallory then forwards the message to Bob using K_MB(\"{msg}\") = {mallory_encrypted}")

        bob_decrypted = self.bob.decrypt_message(mallory_encrypted)
        print(f"Bob decrypted the message to \"{bob_decrypted}\"")

In [117]:
dh = DH_MITM()
dh.key_exchange()

Bob computed key k=1588764742082970607298721048843865523357027938628071447336181923696240186624751913383169706612940347156313875930978230613407398961938704555198526858783030113156846274102676803654202180971298764402950349435593523707276311146871098628485446491251636640019582371596490034350891774359389514192145528482898408239644827561918287385469887947960984587843117786071378317222492270009855459107792619347472129474879842454332313335971285114395380366931247702156158460250416630
Alice computed key k=684538454066020202028106100397350294671108580949516799179882822476760968032407007303996184696774958437167070591223308005811712195121269835227943031253034361090949470163916489750660812153875308817000711130723426425133341220314485525352304166636162312703322768867943496208532784873396014040092741277866193412380333956692004555821233370555927840393707007021566496904026704970395608013738571701205245978908315565479446549880461555707043157665383749183734414249988283


We can see that now Bob and Alice do not have the same keys! Indeed, they each share a key with Mallory, that is in between them. 

![Full MITM DH key exchange diagram](DH_MITM_solved.png)

However, communication between the two parties is still possible when Mallory acts as a proxy, at the cost of her being able to decrypt everything:

In [119]:
dh.communicate("I don't know how it works")

Alice sends K_AM("I don't know how it works") in Bob's direction : b"\x0cIaq\xf6\x9am\xa8Z\x8b\xb9g\xf7O\x11\x9a\x83r\x87\x03\xc1\xcczy\x84\xe1w\xdb\x1a\xfe\xfd\xcf\x88\xb26\x81v'\xb1\xf2\xf5\xb8{~\xf5\x03)\x99)\x1d\x98-:"
Mallory intercepts a message from Alice and decrypts it to "I don't know how it works"
Mallory then forwards the message to Bob using K_MB("I don't know how it works") = b"d\xf3\xe4$\xfc}\xae\xc75]_mVq\xa7\x02\xde8K\xffx>\xe1\x1b\x06\\\x1b\\\x0e\xa8;xr\xd6\xac\xb3\xa7\xd25\xb2\xf5\xa3)^\x03\xa9\xcaT\xab\xb3'\xfd\xe5"
Bob decrypted the message to "I don't know how it works"
